In [ ]:
# Imports
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision import transforms, models, utils, datasets
from torch import nn
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torch.optim as optim

from glob import glob
from PIL import Image
import random

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Inception model
inception = models.inception_v3(pretrained = True)

# freeze model parameters
for param in inception.parameters():
    param.requires_grad = False

# fully connected layer to output 10 classes
num_feats = inception.fc.in_features
inception.fc = nn.Linear(num_feats, 10)

inception = inception.to(DEVICE)

In [ ]:
labels = ['Apple','Kiwi','Banana','Cherry','Orange','Mango','Avocado','Pinenapple','Strawberries']

In [ ]:
# Applying transformation to data inception
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

])

In [ ]:
train_data = datasets.ImageFolder(root = '/content/drive/MyDrive/Data/train', transform = transform)
train_dataloader = DataLoader(train_data, batch_size = 32, shuffle = True)

val_data = datasets.ImageFolder(root = '/content/drive/MyDrive/Data/test', transform = transform)
val_dataloader = DataLoader(val_data, batch_size = 32, shuffle = False)

In [ ]:
# Training Function inception
def train(model, optimizer, train_loader, val_loader, n_epoch, DEVICE):
    criterion = nn.CrossEntropyLoss()
    result = []
    val_res = []

    for e in range(n_epoch):
        model.train()
        correct, total_loss = 0, 0
        total = 0

        for sample, target in train_loader:
            sample, target = sample.to(DEVICE).float(), target.to(DEVICE).long()
            optimizer.zero_grad()
            outputs, aux_outputs = model(sample)
            loss1 = criterion(outputs, target)
            loss2 = criterion(aux_outputs, target)
            loss = loss1 + 0.4*loss2
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

        acc_train = 100 * correct / total
        _, val_acc = valid(model, val_loader, DEVICE)
        print(f'Epoch: [{e+1}/{n_epoch}], Loss: {total_loss / total:.4f}, Train Acc: {acc_train:.2f}, Val Acc: {val_acc:.2f}')

        result.append(acc_train)
        val_res.append(val_acc)

    # Save results to files
    np.savetxt('result.csv', np.array(result), fmt='%.2f', delimiter=',')
    np.savetxt('val_result.csv', np.array(val_res), fmt='%.2f', delimiter=',')


In [ ]:
# Validation Function
def valid(model, val_loader, DEVICE):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sample, target in val_loader:
            sample = sample.to(DEVICE).float()
            target = target.to(DEVICE).long()
            outputs = model(sample)
            _, predicted = torch.max(outputs.data, 1)

            total += target.size(0)
            correct += (predicted == target).sum().item()

    val_accuracy = 100 * correct / total
    return [], val_accuracy

In [ ]:
def plot():
    train_data = np.loadtxt('result.csv', delimiter=',')
    val_data = np.loadtxt('val_result.csv', delimiter=',')
    plt.figure()
    plt.plot(range(1, len(train_data) + 1), train_data, color='blue', label='Train')
    plt.plot(range(1, len(val_data) + 1), val_data, color='red', label='Validation')
    plt.legend()
    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Accuracy (%)', fontsize=14)
    plt.title('Train and Validation Accuracy', fontsize=16)
    plt.savefig('plot.png')
    plt.show()

In [ ]:
optimizer = optim.SGD(inception.fc.parameters(), lr=0.001, momentum=0.9)
train(inception, optimizer, train_dataloader, val_dataloader, n_epoch=10, DEVICE=DEVICE)
plot()

In [ ]:
# Prediction Plots
with torch.no_grad():
    image_paths = glob('/content/drive/MyDrive/Data/predict/*.jpeg')
    random.shuffle(image_paths)
    selected_image_paths = image_paths[:10]
    fig, axes = plt.subplots(2, 5, figsize=(15, 10))
    axes = axes.ravel()

    for i, fpath in enumerate(selected_image_paths):
        image = Image.open(fpath).convert("RGB")
        image_tensor = transform(image).to(DEVICE)
        output = inception(image_tensor.unsqueeze(0))
#         output = resnet(image_tensor.unsqueeze(0))
#         output = mobilenet(image_tensor.unsqueeze(0))
        _, prediction = torch.max(output, 1)
        predicted_label = labels[prediction.cpu().item()]
        axes[i].imshow(image)
        axes[i].set_title(f"Predicted: {predicted_label}")
        axes[i].axis('off')

    for j in range(i+1, 6):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()